In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import glob

from scipy import sparse
import networkx as nx

In [ ]:
pd.set_option('display.max_colwidth', 400)

In [ ]:
n = 108
path_to_img = '../cells'
x = 32
y = 32
inf_val = 999999999999999

In [ ]:
def _make_edges_3d(n_x, n_y, n_z=1):
    vertices = np.arange(n_x * n_y * n_z).reshape((n_x, n_y, n_z))
    edges_deep = np.vstack((vertices[:, :, :-1].ravel(),
                            vertices[:, :, 1:].ravel()))
    edges_right = np.vstack((vertices[:, :-1].ravel(),
                             vertices[:, 1:].ravel()))
    edges_down = np.vstack((vertices[:-1].ravel(), vertices[1:].ravel()))
    edges = np.hstack((edges_deep, edges_right, edges_down))
    return edges

In [ ]:
def _compute_gradient_3d(edges,img):
    _, n_y, n_z = np.array(img).shape
    img = np.array(img)
    gradient = np.abs(
        img[
            edges[0] // (n_y * n_z),
            (edges[0] % (n_y * n_z)) // n_z,
            (edges[0] % (n_y * n_z)) % n_z,
        ]
        - img[
            edges[1] // (n_y * n_z),
            (edges[1] % (n_y * n_z)) // n_z,
            (edges[1] % (n_y * n_z)) % n_z,
        ]
    )
    return gradient

In [ ]:
def _to_graph(
    n_x, n_y, n_z, mask=None, img=None, return_as=sparse.coo_matrix, dtype=None
):
    edges = _make_edges_3d(n_x, n_y, n_z)


    if dtype is None:  
        if img is None:
            dtype = int
        else:
            dtype = img.dtype

    if img is not None:
        img = np.atleast_3d(img)
        weights = _compute_gradient_3d(edges, img)
        if mask is not None:
            edges, weights = _mask_edges_weights(mask, edges, weights)
            diag = img.squeeze()[mask]
        else:
            diag = img.ravel()
        n_voxels = diag.size
    else:
        if mask is not None:
            mask = mask.astype(dtype=bool, copy=False)
            edges = _mask_edges_weights(mask, edges)
            n_voxels = np.sum(mask)
        else:
            n_voxels = n_x * n_y * n_z
        weights = np.ones(edges.shape[1], dtype=dtype)
        diag = np.ones(n_voxels, dtype=dtype)

    diag_idx = np.arange(n_voxels)
    i_idx = np.hstack((edges[0], edges[1]))
    j_idx = np.hstack((edges[1], edges[0]))
    graph = sparse.coo_matrix(
        (
            np.hstack((weights, weights, diag)),
            (np.hstack((i_idx, diag_idx)), np.hstack((j_idx, diag_idx))),
        ),
        (n_voxels, n_voxels),
        dtype=dtype,
    )
    if return_as is np.ndarray:
        return graph.toarray()
    return return_as(graph)

In [ ]:
def img_to_graph(img, *, mask=None, return_as=sparse.coo_matrix, dtype=None):
    img = np.atleast_3d(img)
    n_x, n_y, n_z = img.shape
    return _to_graph(n_x, n_y, n_z, mask, img, return_as, dtype)

In [ ]:
def calc_weight_edges_vertices(val, inputPath, x, y):
    res_1 = []
    res_2 = []

    for i in range(val):
        basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        imagepaths = sorted(list(glob.glob(basePath)))
        metric = []
        pathname = []       
      
        output_1 = []
        output_2 = []
        
        print(basePath)
        print(imagepaths)

        for i in imagepaths:
            img = cv2.imread(i)
            img = cv2.resize(img, (x, y))
            graph = img_to_graph(img)
            arr = graph.toarray()
            G = nx.from_numpy_matrix(arr, parallel_edges=False, create_using=nx.Graph)
            metric.append(G.size('weight')/G.number_of_nodes())   
            pathname.append(i)

        output_1 = metric
        res_1.append(output_1)
        res_ = [item for sublist in res_1 for item in sublist]
        
        
        output_2 = pathname
        res_2.append(output_2)
        res__ = [item for sublist in res_2 for item in sublist]
        
        df = pd.DataFrame(zip(res_,res__), columns=['metric', 'path'])
    
        
    return df

In [ ]:
weight_edges_vertices = calc_weight_edges_vertices(n, path_to_img, x, y)

In [ ]:
weight_edges_vertices.head()

In [ ]:
def calc_weight_edges(val, inputPath, x, y):
    res_1 = []
    res_2 = []

    for i in range(val):
        basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        imagepaths = sorted(list(glob.glob(basePath)))
        metric = []
        pathname = []       
      
        output_1 = []
        output_2 = []
        
        print(basePath)
        print(imagepaths)

        for i in imagepaths:
            img = cv2.imread(i)
            img = cv2.resize(img, (x, y))
            graph = img_to_graph(img)
            arr = graph.toarray()
            G = nx.from_numpy_matrix(arr, parallel_edges=False, create_using=nx.Graph)
            metric.append(G.size('weight')/G.number_of_edges())   
            pathname.append(i)

        output_1 = metric
        res_1.append(output_1)
        res_ = [item for sublist in res_1 for item in sublist]
        
        
        output_2 = pathname
        res_2.append(output_2)
        res__ = [item for sublist in res_2 for item in sublist]
        
        df = pd.DataFrame(zip(res_,res__), columns=['metric', 'path'])
    
        
    return df

In [ ]:
weight_edges = calc_weight_edges(n, path_to_img, x, y)

In [ ]:
weight_edges.head()

In [ ]:
def calc_degree_connect(val, inputPath, x, y):
    res_1 = []
    res_2 = []

    for i in range(val):
        basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        imagepaths = sorted(list(glob.glob(basePath)))
        metric = []
        pathname = []       
      
        output_1 = []
        output_2 = []
        
        print(basePath)
        print(imagepaths)

        for i in imagepaths:
            img = cv2.imread(i)
            img = cv2.resize(img, (x, y))
            graph = img_to_graph(img)
            arr = graph.toarray()
            G = nx.from_numpy_matrix(arr, parallel_edges=False, create_using=nx.Graph)
            metric.append(sum(nx.average_degree_connectivity(G, weight="weight").values()) / len(nx.average_degree_connectivity(G)))   
            pathname.append(i)

        output_1 = metric
        res_1.append(output_1)
        res_ = [item for sublist in res_1 for item in sublist]
        
        
        output_2 = pathname
        res_2.append(output_2)
        res__ = [item for sublist in res_2 for item in sublist]
        
        df = pd.DataFrame(zip(res_,res__), columns=['metric', 'path'])
    
        
    return df

In [ ]:
average_degree_connect = calc_degree_connect(n, path_to_img, x, y)

In [ ]:
average_degree_connect.head()

In [ ]:
def calc_degree_assort(val, inputPath, x, y):
    res_1 = []
    res_2 = []

    for i in range(val):
        basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        imagepaths = sorted(list(glob.glob(basePath)))
        metric = []
        pathname = []       
      
        output_1 = []
        output_2 = []
        
        print(basePath)
        print(imagepaths)

        for i in imagepaths:
            img = cv2.imread(i)
            img = cv2.resize(img, (x, y))
            graph = img_to_graph(img)
            arr = graph.toarray()
            G = nx.from_numpy_matrix(arr, parallel_edges=False, create_using=nx.Graph)
            metric.append(nx.degree_assortativity_coefficient(G, weight="weight"))   
            pathname.append(i)

        output_1 = metric
        res_1.append(output_1)
        res_ = [item for sublist in res_1 for item in sublist]
        
        
        output_2 = pathname
        res_2.append(output_2)
        res__ = [item for sublist in res_2 for item in sublist]
        
        df = pd.DataFrame(zip(res_,res__), columns=['metric', 'path'])
    
        
    return df

In [ ]:
degree_assortativity = calc_degree_assort(n, path_to_img, x, y)

In [ ]:
degree_assortativity.head()

In [ ]:
def calc_local_efficiency(val, inputPath, x, y):
    res_1 = []
    res_2 = []

    for i in range(val):
        basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        imagepaths = sorted(list(glob.glob(basePath)))
        metric = []
        pathname = []       
      
        output_1 = []
        output_2 = []
        
        print(basePath)
        print(imagepaths)

        for i in imagepaths:
            img = cv2.imread(i)
            img = cv2.resize(img, (x, y))
            graph = img_to_graph(img)
            arr = graph.toarray()
            G = nx.from_numpy_matrix(arr, parallel_edges=False, create_using=nx.Graph)
            metric.append(nx.local_efficiency(G))   
            pathname.append(i)

        output_1 = metric
        res_1.append(output_1)
        res_ = [item for sublist in res_1 for item in sublist]
        2
        
        
        
        output_2 = pathname
        res_2.append(output_2)
        res__ = [item for sublist in res_2 for item in sublist]
        
        df = pd.DataFrame(zip(res_,res__), columns=['metric', 'path'])
    
        
    return df

In [ ]:
local_efficiency = calc_local_efficiency(n, path_to_img, x, y)

In [ ]:
local_efficiency.head()

In [ ]:
def calc_global_efficiency(val, inputPath, x, y):
    res_1 = []
    res_2 = []

    for i in range(val):
        basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        imagepaths = sorted(list(glob.glob(basePath)))
        metric = []
        pathname = []       
      
        output_1 = []
        output_2 = []
        
        print(basePath)
        print(imagepaths)

        for i in imagepaths:
            img = cv2.imread(i)
            img = cv2.resize(img, (x, y))
            graph = img_to_graph(img)
            arr = graph.toarray()
            G = nx.from_numpy_matrix(arr, parallel_edges=False, create_using=nx.Graph)
            metric.append(nx.global_efficiency(G))   
            pathname.append(i)

        output_1 = metric
        res_1.append(output_1)
        res_ = [item for sublist in res_1 for item in sublist]
        
        
        output_2 = pathname
        res_2.append(output_2)
        res__ = [item for sublist in res_2 for item in sublist]
        
        df = pd.DataFrame(zip(res_,res__), columns=['metric', 'path'])
    
        
    return df

In [ ]:
global_efficiency = calc_global_efficiency(n, path_to_img, x, y)

In [ ]:
global_efficiency.head()

In [ ]:
def calc_estrada_index(val, inputPath, x, y):
    res_1 = []
    res_2 = []

    for i in range(val):
        basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        imagepaths = sorted(list(glob.glob(basePath)))
        metric = []
        pathname = []       
      
        output_1 = []
        output_2 = []
        
        print(basePath)
        print(imagepaths)

        for i in imagepaths:
            img = cv2.imread(i)
            img = cv2.resize(img, (x, y))
            graph = img_to_graph(img)
            arr = graph.toarray()
            G = nx.from_numpy_matrix(arr, parallel_edges=False, create_using=nx.Graph)
            metric.append(nx.estrada_index(G))   
            pathname.append(i)

        output_1 = metric
        res_1.append(output_1)
        res_ = [item for sublist in res_1 for item in sublist]
        
        
        output_2 = pathname
        res_2.append(output_2)
        res__ = [item for sublist in res_2 for item in sublist]
        
        df = pd.DataFrame(zip(res_,res__), columns=['metric', 'path'])
    
        
    return df

In [ ]:
estrada_index = calc_estrada_index(n, path_to_img, x, y)

In [ ]:
estrada_index.head()

In [ ]:
def calc_wiener_index(val, inputPath, x, y):
    res_1 = []
    res_2 = []

    for i in range(val):
        basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        imagepaths = sorted(list(glob.glob(basePath)))
        metric = []
        pathname = []       
      
        output_1 = []
        output_2 = []
        
        print(basePath)
        print(imagepaths)

        for i in imagepaths:
            img = cv2.imread(i)
            img = cv2.resize(img, (x, y))
            graph = img_to_graph(img)
            arr = graph.toarray()
            G = nx.from_numpy_matrix(arr, parallel_edges=False, create_using=nx.Graph)
            metric.append(nx.wiener_index(G))   
            pathname.append(i)

        output_1 = metric
        res_1.append(output_1)
        res_ = [item for sublist in res_1 for item in sublist]
        
        
        output_2 = pathname
        res_2.append(output_2)
        res__ = [item for sublist in res_2 for item in sublist]
        
        df = pd.DataFrame(zip(res_,res__), columns=['metric', 'path'])
    
        
    return df

In [ ]:
wiener_index = calc_wiener_index(n, path_to_img, x, y)

In [ ]:
wiener_index.head()

In [ ]:
def calc_pagerank(val, inputPath, x, y):
    res_1 = []
    res_2 = []

    for i in range(val):
        basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        imagepaths = sorted(list(glob.glob(basePath)))
        metric = []
        pathname = []       
      
        output_1 = []
        output_2 = []
        
        print(basePath)
        print(imagepaths)

        for i in imagepaths:
            img = cv2.imread(i)
            img = cv2.resize(img, (x, y))
            graph = img_to_graph(img)
            arr = graph.toarray()
            G = nx.from_numpy_matrix(arr, parallel_edges=False, create_using=nx.Graph)
            metric.append(sum(nx.pagerank(G).values())/len(nx.pagerank(G)))   
            pathname.append(i)

        output_1 = metric
        res_1.append(output_1)
        res_ = [item for sublist in res_1 for item in sublist]
        
        
        output_2 = pathname
        res_2.append(output_2)
        res__ = [item for sublist in res_2 for item in sublist]
        
        df = pd.DataFrame(zip(res_,res__), columns=['metric', 'path'])
    
        
    return df

In [ ]:
pagerank = calc_pagerank(n, path_to_img, x, y)

In [ ]:
pagerank.head()

In [ ]:
def calc_degree_cent(val, inputPath, x, y):
    res_1 = []
    res_2 = []

    for i in range(val):
        basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        imagepaths = sorted(list(glob.glob(basePath)))
        metric = []
        pathname = []       
      
        output_1 = []
        output_2 = []
        
        print(basePath)
        print(imagepaths)

        for i in imagepaths:
            img = cv2.imread(i)
            img = cv2.resize(img, (x, y))
            graph = img_to_graph(img)
            arr = graph.toarray()
            G = nx.from_numpy_matrix(arr, parallel_edges=False, create_using=nx.Graph)
            metric.append(sum(nx.degree_centrality(G).values()) / len(nx.degree_centrality(G))) 
            pathname.append(i)

        output_1 = metric
        res_1.append(output_1)
        res_ = [item for sublist in res_1 for item in sublist]
        
        
        output_2 = pathname
        res_2.append(output_2)
        res__ = [item for sublist in res_2 for item in sublist]
        
        df = pd.DataFrame(zip(res_,res__), columns=['metric', 'path'])
    
        
    return df

In [ ]:
degree_centrality = calc_degree_cent(n, path_to_img, x, y)

In [ ]:
degree_centrality.head()

In [ ]:
def calc_avg_neigh_degree(val, inputPath, x, y):
    res_1 = []
    res_2 = []

    for i in range(val):
        basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
        imagepaths = sorted(list(glob.glob(basePath)))
        metric = []
        pathname = []       
      
        output_1 = []
        output_2 = []
        
        print(basePath)
        print(imagepaths)

        for i in imagepaths:
            img = cv2.imread(i)
            img = cv2.resize(img, (x, y))
            graph = img_to_graph(img)
            arr = graph.toarray()
            G = nx.from_numpy_matrix(arr, parallel_edges=False, create_using=nx.Graph)
            metric.append(sum(nx.average_neighbor_degree(G).values())/len(nx.average_neighbor_degree(G)))  
            pathname.append(i)

        output_1 = metric
        res_1.append(output_1)
        res_ = [item for sublist in res_1 for item in sublist]
        
        
        output_2 = pathname
        res_2.append(output_2)
        res__ = [item for sublist in res_2 for item in sublist]
        
        df = pd.DataFrame(zip(res_,res__), columns=['metric', 'path'])
    
        
    return df

In [ ]:
average_neighbor_degree = calc_avg_neigh_degree(n, path_to_img, x,y)

In [ ]:
average_neighbor_degree.head()

In [ ]:
df_invariants = pd.DataFrame(weight_edges_vertices)

In [ ]:
df_invariants = df_invariants.rename(columns={'metric':'weight_edges_vertices'})

In [ ]:
df_invariants['weight_edges'] = weight_edges.metric
df_invariants['average_degree_connectivity'] = average_degree_connect.metric
df_invariants['degree_assortativity'] = degree_assortativity.metric
df_invariants['local_efficiency'] = local_efficiency.metric
df_invariants['global_efficiency'] = global_efficiency.metric
df_invariants['wiener_index'] = wiener_index.metric
df_invariants['estrada_index'] = estrada_index.metric
df_invariants['pagerank'] = pagerank.metric
df_invariants['degree_centrality'] = degree_centrality.metric
df_invariants['average_neighbor_degree'] = average_neighbor_degree.metric

In [ ]:
df_invariants.head()

In [ ]:
df_invariants = df_invariants.replace(np.inf, np.nan)

In [ ]:
df_invariants = df_invariants.fillna(inf_val)

In [ ]:
df_invariants['path_'] = df_invariants.path

In [ ]:
df_invariants[['path', 'label', 'pref', 'img_pref']] = df_invariants['path'].str.split('_',expand=True)

In [ ]:
to_drop = ['path','pref', 'img_pref']

df_invariants = df_invariants.drop(to_drop, axis=1)

In [ ]:
df_invariants.to_excel('invariants_plusminus1.xlsx', index=False)